# Queries for data processing for CI/ASU Decision Theater presentation

Import GEE api and json library. Note that `ee.initialize` will fail if the key for GEE has not yet been setup

In [1]:
import ee
import json

ee.Initialize()

Define a geojson as a string to use for testing

In [2]:
geojson_text = '''{"type": "Polygon", "coordinates": [ [ [ 36.898123159960932, -0.220252698724199 ], [ 37.40818204121706, 0.915787536800825 ], [ 38.613775760549743, 0.011592247301316 ], [ 38.219639352306366, -0.892603042198193 ], [ 36.898123159960932, -0.220252698724199 ] ] ] }'''

## Get statistics on areas degraded within a polygon

In [3]:
def get_degradation_areas(geojson):
    """Return a list of areas within each degradation class for all polygons in geojson"""
    aoi = ee.Geometry(json.loads(geojson))
    "Returns area within each of the five degradation classes for each feature in a geojson"
    # load trends.earth land productivity indicator from assets (original 7 cl, remap to 5 cl)
    te_prod = ee.Image("users/geflanddegradation/global_ld_analysis/r20180821_lp7cl_globe_2001_2015_modis").remap([-32768,1,2,3,4,5,6,7],[-32768,-2,-1,0,0,0,1,2])

    # define the names of the fields
    fields = ["nodata","decline","early_decline","stable","early_improvement","improvement"]

    # multiply pixel area by the area which experienced each of the five transitions --> output: area in ha
    stats = te_prod.eq([-32768,-2,-1,0,1,2]).rename(fields).multiply(ee.Image.pixelArea().divide(10000)).reduceRegions(aoi, ee.Reducer.sum())
    return [feature['properties'] for feature in stats.getInfo()['features']]

get_degradation_areas(geojson_text)

[{u'decline': 864609.2765605276,
  u'early_decline': 226817.09687037364,
  u'early_improvement': 160161.2362287091,
  u'improvement': 114158.3599845793,
  u'nodata': 1297.458236694336,
  u'stable': 658071.179427351}]